## IVFFlat

In [ ]:
"""
IVFFlat
- 역파일 시스템(Inverted File System, IVF)
- 벡터 공간을 여러 보로노이 셀로 분할해서, 벡터들을 그룹화하고 저장하는 방식
- nlist: 보로노이 셀의 개수
- Metric: https://faiss.ai/cpp_api/file/MetricType_8h.html#_CPPv4N5faiss10MetricTypeE
- 셀 배치를 위해서 훈련이 필요함
- nprobe: 쿼리 시에 몇 개의 셀을 방문할지 결정 (index.nprobe=10) 검색 시간은 양자화에 따른 일정한 값과 함께 probe의 수에 대략 선형적으로 증가
- 결론적으로, IndexIVFFlat은 IndexFlatL2에 비해 검색 성능이 향상될 수 있지만, 그 정도의 향상은 여러 요인에 따라 다를 수 있다
"""

## This is too slow, how can I make it faster?

https://github.com/facebookresearch/faiss/wiki/Faster-search

In [ ]:
import numpy as np

In [ ]:
d = 64
nb = 100000
nq = 10000
np.random.seed(1234)

In [ ]:
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [ ]:
xb.shape, xq.shape

In [ ]:
import faiss

In [ ]:
nlist = 100 # 보로노이 셀의 개수
k = 4
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [ ]:
assert not index.is_trained

In [ ]:
# 양자화 구조를 만들기 위해 index를 훈련하기 위한 것
index.train(xb)

In [ ]:
assert index.is_trained

In [ ]:
# add method is where you populate the index with the vectors you want to search 
index.add(xb)

In [ ]:
D, I = index.search(xq, k) # 검색 결과의 거리와 인덱스를 반환
print(I[-5:])

In [ ]:
index.nprobe = 10 # 쿼리 시에 몇 개의 셀을 방문할지 결정
D, I = index.search(xq, k)
print(I[-5:])